In [1]:
def load_data(path, test=False):
    lines = []
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = [l.strip() for l in lines]
    if test:
        ids = [l.split(',')[0] for l in lines]
        return ids, lines
    else:
        return lines

In [2]:
# should we remove dulicates or nah ?
pos = list(set(load_data('data/train_pos_full.txt')))
neg = list(set(load_data('data/train_neg_full.txt')))

#### /!\ faire du data preprocessing ici

In [3]:
# besoin de ça pour train le naive bayes classifier
def extract_features(word_list):
    return dict([(word, True) for word in word_list])

In [4]:
pos_features = [(extract_features(l.split()),'1') for l in pos]
neg_features = [(extract_features(l.split()),'-1') for l in neg]

In [5]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

In [6]:
# on garde un peu de test set pour verifier
threshold_factor = 0.8
pos_thresh = int(threshold_factor * len(pos_features))
neg_thresh = int(threshold_factor * len(neg_features))

In [15]:
train = pos_features+neg_features #pos_features[:pos_thresh]+neg_features[:neg_thresh]
test = pos_features[pos_thresh:]+neg_features[neg_thresh:]

print("Number of training datapoints:", len(train))
print("Number of test datapoints:", len(test))

Number of training datapoints: 2270482
Number of test datapoints: 454097


In [16]:
classifier = NaiveBayesClassifier.train(train)
print("Accuracy of the classifier: ", nltk.classify.util.accuracy(classifier, test))

Accuracy of the classifier:  0.7811172502791254


In [17]:
# en gros les mots qui apparaissent QUE dans les negatifs (ou QUE positifs) sont considérés comme "informative"
classifier.show_most_informative_features()

Most Informative Features
               paperback = True               -1 : 1      =   3331.1 : 1.0
                  a-tech = True               -1 : 1      =   3043.3 : 1.0
               hardcover = True               -1 : 1      =   1805.7 : 1.0
                     ddr = True               -1 : 1      =   1611.3 : 1.0
                  compaq = True               -1 : 1      =   1277.8 : 1.0
            manufactured = True               -1 : 1      =   1118.8 : 1.0
                   512mb = True               -1 : 1      =   1007.4 : 1.0
                  lenovo = True               -1 : 1      =    943.6 : 1.0
             manufacture = True               -1 : 1      =    824.6 : 1.0
               casecrown = True               -1 : 1      =    815.3 : 1.0


In [18]:
# test sur des inputs random
input_reviews = [
    "omg so great i love it !",
    "#blessed justin bieber",
    "worst thing ever just happened",
    "( (",
    "is the best great love amazing justin bieber"
]

for review in input_reviews:
    print("\nReview:", review)
    probdist = classifier.prob_classify(extract_features(review.split()))
    pred_sentiment = probdist.max()
    print("Predicted sentiment: ", pred_sentiment)
    print("Probability: ", round(probdist.prob(pred_sentiment), 2))


Review: omg so great i love it !
Predicted sentiment:  1
Probability:  0.86

Review: #blessed justin bieber
Predicted sentiment:  1
Probability:  0.98

Review: worst thing ever just happened
Predicted sentiment:  -1
Probability:  0.92

Review: ( (
Predicted sentiment:  -1
Probability:  0.93

Review: is the best great love amazing justin bieber
Predicted sentiment:  1
Probability:  0.98


#### predict on test set

In [11]:
test_ids, test_data = load_data('data/test_data.txt', test=True)

In [12]:
def predict(test_data):
    preds = []
    for l in test_data:
        preds.append(classifier.classify(extract_features(l.split())))
    return preds

In [13]:
preds = predict(test_data)

In [14]:
# Id,Prediction
# 1,-1
# 2,1
# 3,-1
# ...
with open('out/prediction_2.csv', 'w') as f:
    f.write('Id,Prediction'+'\n')
    for i,p in zip(test_ids, preds):
        f.write(i+','+p+'\n')